# Group E Assignment 5: NILMTK

## Exercise 1

### Load UKDALE data into memory and print out the metadata

In [ ]:
from nilmtk import DataSet
from nilmtk.utils import print_dict
from nilmtk.timeframe import TimeFrame
import pandas as pd

ukdale = DataSet('./data/ukdale.h5')


#train = DataSet('/path/redd.h5')

### 1.2 Print out Metadata

In [ ]:
print_dict(ukdale.metadata)

### Print out Buildings

In [ ]:
print_dict(ukdale.buildings)

### 1.3 Print out the sub-metered appliances in each building

In [ ]:
for build in ukdale.buildings:
    print("Appliances of Building " +str(build))
    print(ukdale.buildings[build].elec.submeters())
    print("---")

In [ ]:
elec = ukdale.buildings[1].elec

### 1.4 Calculate the total energy consumption for building 1 in kWh

In [ ]:
elec.mains().total_energy()

### 1.5 Print out the type of power for mains and sub-meters

In [ ]:
elec.mains().available_ac_types('power')

In [ ]:
elec.submeters().available_ac_types('power')

## Exercise 2

### 2.1 Timeframed "Fridge Freezer" and "Light" Power Plot

In [ ]:
ukdale_window = DataSet('./data/ukdale.h5')
ukdale_window.set_window(start='2014-04-28', end='2014-04-29')

fridge_meter = ukdale_window.buildings[1].elec['fridge freezer']
light_meter = ukdale_window.buildings[1].elec['light']
elec = ukdale_window.buildings[1].elec

In [ ]:
fridge_meter.plot()
light_meter.plot()

### 2.2 Plot Overall Consumption For that time period

In [ ]:
all_window = next(ukdale_window.buildings[1].elec.load())
all_window.head()

In [ ]:
all_window['power'].plot()

### 2.3 Calculate and plot the energy consumption fraction for each sub-meter

In [ ]:
energy_fraction_per_submeter = elec.submeters().energy_per_meter().transpose().fillna(0)
del energy_fraction_per_submeter['reactive']
active_en = energy_fraction_per_submeter['active']
active_en_frac = active_en/active_en.sum()
active_en_frac.plot(kind="bar", figsize=(15,10), title="Active Energy Fraction")

In [ ]:
apparent_en = energy_fraction_per_submeter['apparent']
apparent_en_frac = apparent_en/apparent_en.sum()
apparent_en_frac.plot(kind="bar", figsize=(15,10), title="Apparent Energy Fraction")

### 2.4 Find appliances of the type “single-phase induction motor”

In [ ]:
elec.select_using_appliances(category='single-phase induction motor')

## Exercise 3

In [ ]:
import time
import pandas as pd
from six import iteritems
import matplotlib.pyplot as plt
import numpy as np

train = DataSet('./data/ukdale.h5')
test = DataSet('./data/ukdale.h5')

In [ ]:
train.set_window(end="24-3-2013")
test.set_window(start="25-3-2013")

train_elec = train.buildings[3].elec
test_elec = test.buildings[3].elec

In [ ]:
train_elec.plot()

In [ ]:
test_elec.mains().plot()

In [ ]:
mains = train_elec.mains()
mains_df = next(mains.load())
mains_df.head()

### Combinatorial Optimisation

In [ ]:
start = time.time()
from nilmtk.disaggregate import CombinatorialOptimisation
from nilmtk.tests.testingtools import data_dir

co = CombinatorialOptimisation()
co.train(train_elec)
end = time.time()
print("Train runtime =", end-start, "seconds.")

In [ ]:
pred = {}
gt= {}

for i, chunk in enumerate(test_elec.mains().load()):
    chunk_drop_na = chunk.dropna()
    pred[i] = co.disaggregate_chunk(chunk_drop_na)
    gt[i]={}
    
    for meter in test_elec.submeters().meters:
        # Only use the meters that we trained on (this saves time!)    
        gt[i][meter] = next(meter.load())
    gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i])}, index=next(iter(gt[i].values())).index).dropna()

In [ ]:
gt_overall = pd.concat(gt)
gt_overall.index = gt_overall.index.droplevel()
pred_overall = pd.concat(pred)
pred_overall.index = pred_overall.index.droplevel()

gt_overall = gt_overall[pred_overall.columns]